In [1]:
%load_ext dotenv
%dotenv

In [2]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

class State(TypedDict):
    messages: Annotated[list[dict], add_messages]
    requirement: Annotated[list[tuple[str, str]], add_messages]
    response: str
    evaluation: str
    code: str
    code_eval: str 
    count: int
    code_eval_count: int
    
graph_builder = StateGraph(State)

In [3]:
import os
from langchain.chat_models import init_chat_model

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("API_VERSION")

llm = init_chat_model(
    "azure_openai:gpt-4o",
    azure_deployment=os.environ["DEPLOYMENT_NAME"],
)

import subprocess
from langchain.tools import tool

@tool
def validate_bicep(code: str) -> str:
    """Validate Bicep code syntax."""
    # Bicepコードの基本的な構文チェックを行う（例として簡単なチェックを実装）
    print ("Validating Bicep code...")
    with open("temp.bicep", "w") as f:
        f.write(code)
    result = subprocess.run(["bicep", "lint", "temp.bicep"], capture_output=True)
    return result

llm_with_tools = llm.bind_tools([validate_bicep])

In [4]:
import sys
def agent1(state:State): #英訳する関数（後でノードとして定義し利用する）
    # query = state['query']
    messages = state['messages']
    
    # Prompt template(書き方2)
    prompt1 = ChatPromptTemplate([
      ('system', 'あなたは、Azure のアーキテクトです、ユーザの実行したいことを実現するために必要なヒアリング項目を1つだけ質問してください。なお、過去の会話履歴はこちらです。{messages}'),
      {"role": "user", "content": "ユーザーの要件を深掘りするための質問をしてください。ただし、質問は一つだけにしてください。分量は短く、簡潔にしてください。"},
    ])

    agent1 = (
        prompt1
        | llm
        | StrOutputParser()
    )
    
    response = agent1.invoke(messages)
    print(response)
    try:
        user_input = input("User: ")
    except:
        sys.exit(1)
    return {"requirement": [response, user_input], "count": state["count"] + 1, "messages": [{"role":"assistant","content":response},{"role": "user", "content": user_input}]}

In [5]:
def evaluator(state:State): #英訳する関数（後でノードとして定義し利用する）
    messages = state['messages']
    if state['count'] >= 10:
        return {"evaluation": "yes"}
    # Prompt template(書き方2)
    prompt = ChatPromptTemplate([
      ('system', 'あなたはAzureのアーキテクトです。Azure 環境構築時のヒアリング項目のリストが十分であれば yes、そうでなければ no と答えてください。'),
      ('user', """{messages}""")
    ])

    agent2 = (
        prompt
        | llm
        | StrOutputParser()
    )

    evaluation = agent2.invoke(messages).lower()
    print(evaluation)
    return {"evaluation": evaluation}

In [6]:
def loop_or_end(state):
    # quality_okがFalseなら"再度意図判定(classify)"へ、それ以外は"end"
    return "agent_1" if not state["evaluation"]=="yes" else "code_gen"

In [7]:
def code_gen(state: State):
    history = [msg for msg in state["messages"] if msg.type in ["human", "ai"]]
    response = llm.invoke(
        [
            {"role": "system", "content": "あなたは優秀な Azure エンジニアです。ユーザーの要件に基づいて、Bicep でコードを生成してください。コードのみを生成し、説明は一切含めないでください。"},
            *history,
            {"role": "user", "content": "上記の要件に基づいて、Bicepでコードを生成してください。"},
        ]
    )
    print('code:', response.content)
    return {"code": response.content}

In [8]:
def code_lint(state:State):
    code = state['code']
    # Prompt template(書き方2)
    prompt = ChatPromptTemplate([
      ('system', 'あなたはAzureのエンジニアです。以下のBicepコードに誤りがないか確認し、問題があれば again、なければ done と答えてください。'),
      ('user', """{code}""")
    ])

    agent3 = (
        prompt
        | llm_with_tools
        | StrOutputParser()
    )

    response = agent3.invoke(code).lower()
    print('code_eval', response)
    return {"code_eval": response}
  
def loop_or_end_code(state):
  # quality_okがFalseなら"再度意図判定(classify)"へ、それ以外は"end"
  return "code_gen" if not state["code_eval"]=="done" else "end"

In [9]:
workflow = StateGraph(State)

workflow.add_node("agent_1", agent1)
workflow.add_node("evaluator", evaluator)
workflow.add_node("code_gen", code_gen)
workflow.add_node("code_lint", code_lint)

workflow.add_edge(START, "agent_1")
workflow.add_edge("agent_1", "evaluator")
workflow.add_conditional_edges("evaluator", loop_or_end, {
    "agent_1": "agent_1",
    "code_gen": "code_gen",
})
workflow.add_edge("code_gen", "code_lint")
workflow.add_conditional_edges("code_lint", loop_or_end_code, {
    "code_gen": "code_gen",
    "end": END
})

graph_app = workflow.compile()

In [10]:
initial_state = State(query='Azure上にシステムを構築したい', count=0) #Agentに翻訳してほしい言葉を入れる

In [11]:
# streamを利用する場合
for chunk in graph_app.stream(initial_state):
    print(chunk)

システムの目的と達成したいビジネスゴールは何ですか？
{'agent_1': {'requirement': ['システムの目的と達成したいビジネスゴールは何ですか？', 'WEB'], 'count': 1, 'messages': [{'role': 'assistant', 'content': 'システムの目的と達成したいビジネスゴールは何ですか？'}, {'role': 'user', 'content': 'WEB'}]}}
no
{'evaluator': {'evaluation': 'no'}}
対象ユーザー層と想定する利用シナリオは何ですか？
{'agent_1': {'requirement': ['対象ユーザー層と想定する利用シナリオは何ですか？', '社内向け'], 'count': 2, 'messages': [{'role': 'assistant', 'content': '対象ユーザー層と想定する利用シナリオは何ですか？'}, {'role': 'user', 'content': '社内向け'}]}}
no
{'evaluator': {'evaluation': 'no'}}
具体的にどのような機能が必要ですか？
{'agent_1': {'requirement': ['具体的にどのような機能が必要ですか？', 'Applicationのホスト'], 'count': 3, 'messages': [{'role': 'assistant', 'content': '具体的にどのような機能が必要ですか？'}, {'role': 'user', 'content': 'Applicationのホスト'}]}}
no
{'evaluator': {'evaluation': 'no'}}
アプリケーションの規模と必要なリソース量はどの程度ですか？
{'agent_1': {'requirement': ['アプリケーションの規模と必要なリソース量はどの程度ですか？', '最小限'], 'count': 4, 'messages': [{'role': 'assistant', 'content': 'アプリケーションの規模と必要なリソース量はどの程度ですか？'}, {'role': 'user', 'content': '最小限

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass